# Model 6
using the best hyperparameters from the previous version. testing on new data.
new data include new columns for each categorical column with a dominant category
if the column value has the dominant value, then there is a 1 corresponding to "isDominant"
other wise it is set to 0.
Also adding a Linear Regression

## Results:
The linear regression shows no improvement on xgboost. 
With adding new data and applying feature engineering, the RMSE did not improve 

** RMSE Value 0.726979392992975**

In [1]:
# reading data from google drive 
# data paths whether notebook is run locally or google colab
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COMPETETION_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1"
    TRAIN_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/train.csv"
    TEST_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/test.csv"
    OUTPUT_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/output"
except:
    TRAIN_DATA_PATH = os.path.join("data", "train.csv")
    TEST_DATA_PATH = os.path.join("data", "test.csv")

print(f"Training Path {TRAIN_DATA_PATH}")
print(f"Testing Path {TEST_DATA_PATH}")



Training Path data/train.csv
Testing Path data/test.csv


In [18]:

# library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# modeling
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

# evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
def read_organise_data(train=TRAIN_DATA_PATH):
    """read the data from a path and splitting to features and target

    Args:
        train (path, optional): The path of training data file to. Defaults to TRAIN_DATA_PATH.

    Returns:
        X, y: X for features and y for target
    """
    full_df = pd.read_csv(train, index_col="id")
    print(f"Shape of Dataset: {full_df.shape}")
    return full_df

df= read_organise_data()
X_test = pd.read_csv(TEST_DATA_PATH, index_col='id')
print(f"Shape of Test set {X_test.shape}")

Shape of Dataset: (300000, 25)
Shape of Test set (200000, 24)


## Understanding Data

In [5]:
y = df.target.copy()
X = df.drop('target', axis=1).copy()

In [6]:
# splitting the data
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2,
                                                      random_state= 1)

In [7]:

# categorical columns start with cat
cat_cnames = [cname for cname in X.columns if 'cat' in cname]
# numerical columns starts with cont
num_cnames = [cname for cname in X.columns if 'cont' in cname]
total_cols = cat_cnames + num_cnames
print(len(total_cols))

24


In [8]:

# checking the cardinality of the categorical columns
for cname in cat_cnames:
    num_unique = X[cname].nunique()
    print(f"{cname} has {num_unique}")
    if num_unique > 10:
        print(f"\t{cname} has a high cardinality")

# eventhough cat9 col has more than 10 unique values, we will still use the OnehotEncoder 

cat0 has 2
cat1 has 2
cat2 has 2
cat3 has 4
cat4 has 4
cat5 has 4
cat6 has 8
cat7 has 8
cat8 has 7
cat9 has 15
	cat9 has a high cardinality


In [9]:
## columns with a dominant category
# category columns with 1 value of more than 90% of data 
one_value_cat = []
for cname in cat_cnames:
    if X[cname].value_counts().iloc[0]/len(df) > 0.90:
        one_value_cat.append(cname)

one_value_cat

['cat4', 'cat6', 'cat7']

In [10]:
# adding a column corresponding to wethere a value is dominant or not
# for each of the columns with 1 dominant value, add a column with the column name as a prefix
# new columns: the values are 1 if the the row value of the column belongs to dominant category and 0 if the column value is not the dominant category
# 
for cname in one_value_cat:
    most_frequent_value = X[cname].value_counts().sort_values(ascending=False).index[0]
    X[f'{cname}_isDominante'] = X[cname].apply(lambda x: 0 if x!=most_frequent_value else 1)
    X_test[f'{cname}_isDominante'] = X_test[cname].apply(lambda x: 0 if x!=most_frequent_value else 1)

In [11]:
X.shape

(300000, 27)

In [12]:
X_test.shape

(200000, 27)

In [13]:

# numerical and categorical transformations
num_transformer = StandardScaler()
cat_transformer = OrdinalEncoder()
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cnames),
    ('cat', cat_transformer, cat_cnames),
],remainder='passthrough')

In [14]:
## pipeline

best_model = XGBRegressor(n_estimators=1350, learning_rate=0.01, max_depth=3,
                          objective='reg:squarederror', random_state=1)

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])


In [57]:
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('cat',
               

In [62]:

prediction_valid = my_pipeline.predict(X_valid)
print(mean_squared_error(y_valid, prediction_valid)**(1/2))
# old value before adding new columns prediction_valid = my_pipeline.predict(X_valid)
#print(mean_squared_error(y_valid, prediction_valid)**(1/2))
# old value before adding new columns 0.7178614528083396

0.726979392992975


In [39]:
predictions = best_pipeline.predict(X_test)

In [40]:
OUTPUT_PATH = 'output'
def output_submission(prediction, file_name):
    """creating a kaggle submission file

    Args:
        prediction (array): an array of predictions of the test dataset
        file_name (string): a string for the name without the extension
    """
    my_submission = pd.DataFrame({'target': predictions},
                                 index=X_test.index)
    #my_submission.set_index('id')
    file_path = os.path.join(OUTPUT_PATH,file_name)
    my_submission.to_csv(f'{file_path}.csv')
    print(f'A submission file has been made at {file_path}')

In [41]:
output_submission(predictions, "Submission10")

A submission file has been made at output/Submission10


In [15]:
# linear regression pipeline
linear_model = LinearRegression()
linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_model),
])

In [16]:
linear_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('cat',
               

In [20]:
mean_squared_error(y_valid, linear_pipeline.predict(X_valid))**(1/2)

0.7372876297579729

In [22]:
y2 = df.target.copy()
X2 = df.drop('target', axis=1).copy()

In [24]:
X_train2, X_valid2, y_train2, y_valid2 = train_test_split(X2, y2, test_size=0.2, random_state=1)

In [25]:
## pipeline

best_model2 = XGBRegressor(n_estimators=1350, learning_rate=0.01, max_depth=3,
                          objective='reg:squarederror', random_state=1)

my_pipeline2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model2)
])


In [26]:
my_pipeline2.fit(X_train2, y_train2)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('cat',
               

In [28]:
xgb_predictions2 = my_pipeline2.predict(X_valid2)
rmse = mean_squared_error(y_valid2, xgb_predictions2)
print(f"Root Mean Squeared Error {rmse**(1/2)}")

Root Mean Squeared Error 0.726979392992975


In [31]:
# linear regression pipeline
linear_model2 = LinearRegression()
linear_pipeline2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_model2),
])
linear_pipeline2.fit(X_train2, y_train2)
linear_predictions2 = linear_pipeline2.predict(X_valid2)
linear_rmse = mean_squared_error(y_valid2, linear_predictions2)
print(f"Linear Model on original data: {linear_rmse**(1/2)}")

Linear Model on original data: 0.7372876297579729


In [32]:
X.shape

(300000, 27)

In [33]:
X2.shape

(300000, 24)